# project ai: Easer

by Michiel Téblick and thibaut Van Goethem

In this notebook we will look at the easer model proposed at https://dl.acm.org/doi/pdf/10.1145/3308558.3313710.

This model will be applied to a dataset from foods.com which containes a bunch of recipes with user ratings/reactions on them.

In [1]:
import math
from copy import deepcopy
import pandas as pd
import numpy as np
import scipy
import bottleneck as bn
from sklearn.model_selection import KFold
import time
import pickle
from scipy import sparse
import statistics as st

## Reading and preprocessing the data

In [2]:
use_less_data = True


df_train = pd.read_csv('../folds/fold_0/train.csv')
df_test = pd.read_csv('../folds/fold_0/test.csv')
df_validate = pd.read_csv('../folds/fold_0/validate.csv')
df = pd.concat([df_train, df_test, df_validate])
#df.reset_index()

print("amount of interactions in the full dataset: ",len(df))
print("amount of recipes in the full dataset: ",len(df.recipe_id.unique()))
print("amount of users in the full dataset: ",len(df.user_id.unique()))

if use_less_data:
    df = df[df['count_item'] >= 10]
    print("amount of recipes in the smaller dataset: ",len(df.recipe_id.unique()))
    print("amount of users in the smaller dataset: ",len(df.user_id.unique()))
df.reset_index()


amount of interactions in the full dataset:  733951
amount of recipes in the full dataset:  80511
amount of users in the full dataset:  32635
amount of recipes in the smaller dataset:  16577
amount of users in the smaller dataset:  31887


,index,user_id,recipe_id,date,rating,review,count_user,count_item
0,0,56680,79222,2006-11-11,5,"Oh, This was wonderful! Had a soup and salad ...",174,18
1,1,827374,79222,2010-11-29,3,We made this last night and really enjoyed it....,10,18
2,10,61995,195977,2008-10-28,5,I have been making this for more than 10 years...,41,10
3,20,305531,426090,2017-08-20,5,We really enjoyed these beans. Simple but good...,2195,14
4,21,89831,33096,2004-03-15,5,Merlot...this is the second time that I made y...,2572,27
...,...,...,...,...,...,...,...,...
434723,151900,1423741,39902,2012-07-25,4,I switched out the American Cheese for Sharp C...,7,15
434724,151907,422893,196735,2009-01-02,5,"Yum, Yum, I love potatoes with Rosemary & this...",1130,17
434725,151908,96177,196735,2009-01-12,5,We just loved these tatters. Quick easy and ve...,563,17
434726,151909,573325,196735,2010-09-08,5,"What a great, healthy, easy and yummy recipe!<...",880,17


Set all ratings to 1 (even negative interactions are seen as interactions)

In [3]:
df.loc[:,'rating'] = 1

Below here are two ways to cut down on the amount of interactions that are used in this notebook
- The first one randomly removes x% of the users,
- The second one removes all user and recipes that have less than X amount of interaction containing them

We opted for the second form as this is more representative of how the models should be used due to the lower amount if recipes but more reactions per recipe. Also the second choice is a deterministic way of removing data, which the first one is not.
This does end up mostly giving slightly worse result compared to the first choice.

The reason we need to remove data is because a matrix inversion is done, which can not be done in a smart way.
Also the result of the inversion is not necessarily a sparse matrix so the full calculation needs to be done on dense matrices. This end up scaling O(n^3) in time complexity and O(n^2) for memory needed. n here is the amount of recipes.
So running on the full dataset would require more than 200gb of ram which we do not have.


### rescaling the id's
The recipes and users don't go from 0 to amount so if we were to put this in a matrix we would get empty columns and rows. This is not that handy so we reindex both the user_id and recipe_ids

This is a step we must not forget when entering the data in the model, as we also need to remap our input data using the same remapping that was used here

In [4]:
userSet = set(df['user_id'].to_list())
user_transform_dict = dict(map(reversed, enumerate(userSet)))
recipeSet = set(df['recipe_id'].to_list())
recipe_transform_dict = dict(map(reversed, enumerate(recipeSet)))
recipe_dict = dict(enumerate(recipeSet))

In [5]:
keep_nan_user = [k for k, v in user_transform_dict.items() if pd.isnull(v)]
keep_nan_recipe = [k for k, v in recipe_transform_dict.items() if pd.isnull(v)]


def transform_id(dataframe):
    tochange = dataframe['user_id']
    dataframe['user_id'] = tochange.map(user_transform_dict).fillna(tochange.mask(tochange.isin(keep_nan_user)))

    tochange = dataframe['recipe_id']
    dataframe['recipe_id'] = tochange.map(recipe_transform_dict).fillna(tochange.mask(tochange.isin(keep_nan_recipe)))
    return dataframe

def open_csv(filename, use_less_data=False):
    df = pd.read_csv(filename)
    if use_less_data:
        df = df[df['count_item'] >= 10]
    df = transform_id(df)
    df.loc[:,'rating'] = 1
    df.drop('review', axis=1, inplace=True)
    #df.drop('date', axis=1, inplace=True)
    return df


### Creation of the folds


In [6]:
k = 10
folds = list()
for directory in ["../folds/fold_%d" % i for i in range(k)]:
    folds.append((directory + "/test.csv", directory + "/train.csv", directory + "/validate.csv"))

## Creation model
Here we define the models used for the experiments. Both the easer predictor and a populaliry predictor are created. the popularity predictor is used as a baseline

In [7]:
def split_test(data_set):
    ground_truth = data_set.sort_values('date').groupby('user_id').tail(1)
    predict = pd.concat([data_set, ground_truth]).drop_duplicates(keep=False)
    return predict, ground_truth

def data_frame_to_matrix(dataframe):
    ratings = dataframe.rating
    idx = (dataframe.user_id, dataframe.recipe_id)
    return sparse.csc_matrix((ratings, idx), shape=(len(df.user_id.unique()), len(df.recipe_id.unique())),
                                dtype=float)

## Creation model
Here we define the models used for the experiments. Both the easer predictor and a populaliry predictor are created. the popularity predictor is used as a baseline

In [8]:
alpha = 0.75
def precompute(train_data):
    userCount = train_data.shape[0]
    XtX= np.asarray(train_data.T.dot(train_data).todense(), dtype = np.float32)
    #del train_data  # only the item-item data-matrix XtX is needed in the following

    mu=np.diag(XtX) / userCount   # the mean of the columns in train_data (for binary train_data)
    variance_times_userCount = np.diag(XtX) - mu * mu * userCount # variances of columns in train_data (scaled by userCount)

    # standardizing the data-matrix XtX (if alpha=1, then XtX becomes the correlation matrix)
    XtX -= mu[:,None] *(mu* userCount)
    rescaling = np.power(variance_times_userCount, alpha / 2.0)
    scaling = 1.0  / rescaling
    XtX = scaling[:,None] * XtX * scaling

    XtXdiag = deepcopy(np.diag(XtX))
    ii_diag = np.diag_indices(XtX.shape[0])

    # print("number of items: {}".format(len(mu)))
    # print("number of users: {}".format(userCount))
    return XtX,XtXdiag,ii_diag,rescaling,scaling

def precompute_sparse(train_data):
    userCount = train_data.shape[0]
    XtX= train_data.T.dot(train_data)
    #del train_data  # only the item-item data-matrix XtX is needed in the following

    mu=XtX.diagonal() / userCount
    variance_times_userCount = XtX.diagonal() - mu * mu * userCount

    # standardizing the data-matrix XtX (if alpha=1, then XtX becomes the correlation matrix)
    XtX -= scipy.sparse.csr_matrix(mu[:,None]) * scipy.sparse.csr_matrix(mu* userCount)
    rescaling = np.power(variance_times_userCount, alpha / 2.0)
    scaling = 1.0  / rescaling

    XtX = scipy.sparse.diags(scaling) * XtX * scipy.sparse.diags(scaling)

    XtXdiag = deepcopy(XtX.diagonal())
    ii_diag = np.diag_indices(XtX.shape[0])

    # print("number of items: {}".format(len(mu)))
    # print("number of users: {}".format(userCount))
    return XtX,XtXdiag,ii_diag,rescaling,scaling

In [9]:
class sparseMRF:
    def __init__(self,xtx,xtxdiag,ii,rescaling,scaling):
        self.XtX=xtx
        self.XtXdiag = xtxdiag
        self.ii_diag = ii
        self.threshold = 0.375    # results in sparsity 0.1 % (for alpha=0.75)
        #threshold = 0.11    # results in sparsity 0.5 % (for alpha=0.75)
        self.maxInColumn = 1000
        # hyper-parameter r in the paper, which determines the trade-off between approximation-accuracy and training-time
        self.rr = 0.5
        # L2 norm regularization
        self.L2reg = 1.0
        self.scaling=scaling
        self.rescaling=rescaling
    def sparse_solution(self):

        # sparsity pattern, see section 3.1 in the paper
        self.XtX[self.ii_diag] = XtXdiag
        AA = self.calculate_sparsity_pattern()

        # parameter-estimation, see section 3.2 in the paper
        self.XtX[self.ii_diag] = self.XtXdiag+self.L2reg
        BBsparse = self.sparse_parameter_estimation(AA)

        return BBsparse


    def calculate_sparsity_pattern(self):
        # this implements section 3.1 in the paper.

        # print("sparsifying the data-matrix (section 3.1 in the paper) ...")
        # apply threshold
        # ix = np.where( np.abs(self.XtX) > self.threshold)
        # AA= sparse.csc_matrix( (self.XtX[ix], ix), shape=self.XtX.shape, dtype=np.float32)
        idx1, idx2, value = sparse.find(np.abs(self.XtX > self.threshold))
        boolean_matrix = scipy.sparse.csc_matrix((value, (idx1, idx2)), dtype=np.float32)
        AA = boolean_matrix.multiply(self.XtX)

        # enforce maxInColumn, see section 3.1 in paper
        countInColumns=AA.getnnz(axis=0)
        iiList = np.where(countInColumns > self.maxInColumn)[0]
        # print("    number of items with more than {} entries in column: {}".format(self.maxInColumn, len(iiList)) )
        for ii in iiList:
            jj= AA[:,ii].nonzero()[0]
            kk = bn.argpartition(-np.abs(np.asarray(AA[jj,ii].todense()).flatten()), self.maxInColumn)[self.maxInColumn:]
            AA[  jj[kk], ii ] = 0.0
        AA.eliminate_zeros()
        # print("    resulting sparsity of AA: {}".format( AA.nnz*1.0 / AA.shape[0] / AA.shape[0]) )

        return AA

    def sparse_parameter_estimation(self, AA):
        # this implements section 3.2 in the paper

        # list L in the paper, sorted by item-counts per column, ties broken by item-popularities as reflected by np.diag(XtX)
        AAcountInColumns = AA.getnnz(axis=0)
        sortedList=np.argsort(AAcountInColumns+ self.XtX.diagonal() /2.0/ np.max(self.XtX.diagonal()))[::-1]

        # print("iterating through steps 1,2, and 4 in section 3.2 of the paper ...")
        todoIndicators=np.ones(AAcountInColumns.shape[0])
        blockList=[]   # list of blocks. Each block is a list of item-indices, to be processed in step 3 of the paper
        for ii in sortedList:
            if todoIndicators[ii]==1:
                nn, _, vals=sparse.find(AA[:,ii])  # step 1 in paper: set nn contains item ii and its neighbors N
                kk=np.argsort(np.abs(vals))[::-1]
                nn=nn[kk]
                blockList.append(nn) # list of items in the block, to be processed in step 3 below
                # remove possibly several items from list L, as determined by parameter rr (r in the paper)
                dd_count=max(1,int(np.ceil(len(nn)*self.rr)))
                dd=nn[:dd_count] # set D, see step 2 in the paper
                todoIndicators[dd]=0  # step 4 in the paper

        # print("now step 3 in section 3.2 of the paper: iterating ...")
        # now the (possibly heavy) computations of step 3:
        # given that steps 1,2,4 are already done, the following for-loop could be implemented in parallel.
        BBlist_ix1, BBlist_ix2, BBlist_val = [], [], []
        denseXtX = self.XtX.toarray()
        for nn in blockList:
            #calculate dense solution for the items in set nn
            BBblock=np.linalg.inv(denseXtX[np.ix_(nn,nn)])
            BBblock/=-np.diag(BBblock)
            # determine set D based on parameter rr (r in the paper)
            dd_count=max(1,int(np.ceil(len(nn)*self.rr)))
            dd=nn[:dd_count] # set D in paper
            # store the solution regarding the items in D
            blockix = np.meshgrid(dd,nn)
            BBlist_ix1.extend(blockix[1].flatten().tolist())
            BBlist_ix2.extend(blockix[0].flatten().tolist())
            BBlist_val.extend(BBblock[:,:dd_count].flatten().tolist())

        del denseXtX

        # print("final step: obtaining the sparse matrix BB by averaging the solutions regarding the various sets D ...")
        BBsum = sparse.csc_matrix( (BBlist_val,  (BBlist_ix1, BBlist_ix2  )  ), shape=self.XtX.shape, dtype=np.float32)
        BBcnt = sparse.csc_matrix( (np.ones(len(BBlist_ix1), dtype=np.float32),  (BBlist_ix1,BBlist_ix2  )  ), shape=self.XtX.shape, dtype=np.float32)
        b_div= sparse.find(BBcnt)[2]
        b_3= sparse.find(BBsum)
        BBavg = sparse.csc_matrix( ( b_3[2] / b_div   ,  (b_3[0],b_3[1]  )  ), shape=self.XtX.shape, dtype=np.float32)
        BBavg[self.ii_diag]=0.0

        # print("forcing the sparsity pattern of AA onto BB ...")
        BBavg = sparse.csr_matrix( ( np.asarray(BBavg[AA.nonzero()]).flatten(),  AA.nonzero() ), shape=BBavg.shape, dtype=np.float32)

        # print("    resulting sparsity of learned BB: {}".format( BBavg.nnz * 1.0 / AA.shape[0] / AA.shape[0]) )

        return BBavg

    def train(self):
        # print("training the sparse model:\n")
        BBsparse = self.sparse_solution()
        # print("\ntotal training time (including the time for determining the sparsity-pattern):")
        #
        # print("\nre-scaling BB back to the original item-popularities ...")
        # assuming that mu.T.dot(BB) == mu, see Appendix in paper
        BBsparse=sparse.diags(scaling).dot(BBsparse).dot(sparse.diags(rescaling))

        # print("\nfor the evaluation below: converting the sparse model into a dense-matrix-representation ...")
        self.BB = BBsparse
    def predict(self,predict_matrix):
        return predict_matrix.dot(self.BB)

## training models + k-fold validation

In [10]:
K = 20
K2 = 50

def recal_easer(model, predict_data, test_data):
    total = len(test_data)

    X_train = data_frame_to_matrix(predict_data)
    y_pred = model.predict(X_train).toarray()

    X_test = data_frame_to_matrix(test_data)

    interacted_recipes = (X_train == 1).toarray()
    y_pred[interacted_recipes] = -100000
    idx_top_scores = np.asarray((-y_pred).argsort()[:,:100])
    del y_pred
    dense_X_test = X_test.toarray()

    correct_K = 0
    correct_K2 = 0
    ndcg = 0

    for idx, row in enumerate(idx_top_scores):
        for rank, index in enumerate(row):
            if dense_X_test[idx][index] == 1:
                if rank < K:
                    correct_K += 1
                if rank < K2:
                    correct_K2 += 1
                ndcg += 1/(math.log2(rank+2))

    print("easer recall@%s = %s" % (str(K), str(correct_K / total)))
    print("easer recall@%s = %s" % (str(K2), str(correct_K2 / total)))
    print("easer ndcg@%s = %s" % (100, str(ndcg / total)), end="\n\n")

    return correct_K/total, correct_K2/total, ndcg/total


## training models + k-fold validation


In [11]:
#Please enter the path here of where you will place the pickle files (with trailing /)
data_path="../results_aiproject/"
for f_idx, fold_files in enumerate(folds):
    start = time.time()
    train_data = open_csv(fold_files[0], True)
    #Here we have the user item matrix
    X_train = data_frame_to_matrix(train_data)

    #train models

    # model_pop=popularity()
    # model_pop.train(train_data)
    # modelpopfile = open(data_path+"model_pop_fold" + str(f_idx) + ".pkl", mode='wb')
    # pickle.dump(model_pop, modelpopfile)
    # modelpopfile.close()
    # model_pop = None

    highest_recall = (0,0,0)
    best_lambda = 0
    validate_data = open_csv(fold_files[1], use_less_data)
    newstart=start
    for l in range(100, 1500, 100):
        xtx,XtXdiag,ii_diag,rescaling,scaling=precompute_sparse(X_train)
        xtx2,XtXdiag2,ii_diag2,rescaling2,scaling2=precompute(X_train)
        model=sparseMRF(xtx,XtXdiag,ii_diag,rescaling,scaling)
        model.train()
        print("done training parameter: ", l)
        interactions, ground_truth = split_test(validate_data)

        modelfile = open(data_path+"model_fold" + str(f_idx) + ".pkl", mode='wb')
        pickle.dump(model, modelfile)
        modelfile.close()

        recall20, recall50, ndcg = recal_easer(model, interactions, ground_truth)
        if recall20 > highest_recall[0]:
            highest_recall = (recall20, recall50, ndcg)
            best_lambda = l

            modelfile = open(data_path+"model_fold" + str(f_idx) + ".pkl", mode='wb')
            pickle.dump(model, modelfile)
            modelfile.close()

        end = time.time()
        print("fold took : ", end - newstart, "s")
        newstart=end

    print("done fold:",str(f_idx))

    print("Best parameter lambda: ", best_lambda)
    print("easer fold: %s, recall@%s = %s" % (str(f_idx), str(K), highest_recall[0]))
    print("easer fold: %s, recall@%s = %s" % (str(f_idx), str(K2), highest_recall[1]))
    print("easer fold: %s, ndcg@%s = %s" % (str(f_idx), 100, highest_recall[2]), end="\n\n")

    end = time.time()
    print("training took : ", end - start, "s")
    break



<ipython-input-8-30f462b8f82b>:34: RuntimeWarning: divide by zero encountered in true_divide
  scaling = 1.0  / rescaling
<ipython-input-8-30f462b8f82b>:13: RuntimeWarning: divide by zero encountered in true_divide
  scaling = 1.0  / rescaling
<ipython-input-8-30f462b8f82b>:14: RuntimeWarning: invalid value encountered in multiply
  XtX = scaling[:,None] * XtX * scaling
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


done training parameter:  100
saved
easer recall@20 = 0.010704048728054461
easer recall@50 = 0.016391974202794698
easer ndcg@100 = 0.007988050276139585

fold took :  264.875864982605 s
done fold: 0
Best parameter lambda:  100
easer fold: 0, recall@20 = 0.010704048728054461
easer fold: 0, recall@50 = 0.016391974202794698
easer fold: 0, ndcg@100 = 0.007988050276139585

training took :  264.88905692100525 s


## Evaluation results of the folds

Here we use recall@20, recal@50 and ndcg@100


In [ ]:
result_list_K = list()
result_list_K2 = list()
result_ndcg = list()

for i in range(k):

    #Evaluate recall@k

    data = open_csv(folds[k][2])
    model = pickle.load(open(data_path+"model_fold"+str(i)+".pkl", mode='rb'))
    interactions, ground_truth = split_test(data)

    recall20, recall50, ndcg = recal_easer(model, interactions, ground_truth)

    result_list_K.append(recall20)
    result_list_K2.append(recall50)
    result_ndcg.append(ndcg)

    print("easer fold: %s, recall@%s = %s" % (str(i), str(K), recall20))
    print("easer fold: %s, recall@%s = %s" % (str(i), str(K2), recall50))
    print("easer fold: %s, ndcg@%s = %s" % (str(i), 100, ndcg), end="\n\n")

print("mean recall@%s over 10 folds: " % str(K), str(st.mean(result_list_K)))
print("mean recall@%s over 10 folds: " % str(K2), str(st.mean(result_list_K2)))
print("mean ndcg@%s over 10 folds: " % str(100), str(st.mean(result_ndcg)), end="\n\n")
print("standard deviation recall@%s over 10 folds: " % str(K), str(st.pstdev(result_list_K)))
print("standard deviation recall@%s over 10 folds: " % str(K2), str(st.pstdev(result_list_K2)))
print("standard deviation ndcg@%s over 10 folds: " % str(100), str(st.pstdev(result_ndcg)))

In [ ]:
#recall score for popularity
result_list_pop_K=list()
result_list_pop_K2=list()
result_list_pop_ndcg=list()
for i in range(k):
    data = pickle.load(open(data_path+"data_fold"+str(i)+".pkl", mode='rb'))
    model = pickle.load(open(data_path+"model_pop_fold"+str(i)+".pkl", mode='rb'))
    test_data = data[1]
    predict_data = data[0]
    pop=model.predict()
    total = 0
    correct_K = 0
    correct_K2 = 0
    ndcg = 0
    for idx, interaction in test_data.iterrows():
        user = interaction['user_id']
        user_data = predict_data.loc[(predict_data['user_id'] == user)]
        already_interacted_recipes = user_data[user_data.columns[1]].to_numpy()
        newpop = pop[:150]
        newpop = newpop[~np.in1d(newpop,already_interacted_recipes)]
        newpop_K = newpop[:K]
        newpop_K2 = newpop[:K2]
        newpop_ndcg = newpop[:100]
        recipe = interaction['recipe_id']
        if recipe in newpop_K:
            correct_K += 1
        if recipe in newpop_K2:
            correct_K2 += 1
        if recipe in newpop_ndcg:
            ndcg += 1/(math.log2(np.where(newpop_ndcg == recipe)[0]+2))
        total += 1
    result_list_pop_K.append(correct_K / total)
    result_list_pop_K2.append(correct_K2 / total)
    result_list_pop_ndcg.append(ndcg / total)
    print("popularity fold: %s, recall@%s = %s" % (str(i),str(K), str(correct_K / total)))
    print("popularity fold: %s, recall@%s = %s" % (str(i),str(K2), str(correct_K2 / total)))
    print("popularity fold: %s, ndcg@%s = %s" % (str(i),str(100), str(ndcg / total)), end="\n\n")

print("mean recall@%s over 10 folds: " % str(K), str(st.mean(result_list_pop_K)))
print("mean recall@%s over 10 folds: " % str(K2), str(st.mean(result_list_pop_K2)))
print("mean ndcg@%s over 10 folds: " % str(100), str(st.mean(result_list_pop_ndcg)), end="\n\n")
print("standard deviation recall@%s over 10 folds: " % str(K), str(st.pstdev(result_list_pop_K)))
print("standard deviation recall@%s over 10 folds: " % str(K2), str(st.pstdev(result_list_pop_K2)))
print("standard deviation ndcg@%s over 10 folds: " % str(100), str(st.pstdev(result_list_pop_ndcg)))

The next section is a demonstration that selects a random user and makes a recommendation prediction for this user.

In [ ]:
import random
# read recipe data and load pre-trained model
df_recipes = pd.read_csv('../data/RAW_recipes.csv')
df_recipes.drop(['minutes', 'contributor_id', 'submitted', 'tags',
                 'nutrition', 'n_steps', 'steps', 'description', 'n_ingredients'], axis=1, inplace=True)
data = pickle.load(open(data_path+"data_fold0.pkl", mode='rb'))
model = pickle.load(open(data_path+"model_fold0.pkl", mode='rb'))
predict_data = data[0]
ratings = predict_data.rating
idx = (predict_data.user_id, predict_data.recipe_id)
x_train = sparse.csc_matrix((ratings, idx), shape=(len(df.user_id.unique()), len(df.recipe_id.unique())), dtype=float)

# get random user and make prediction
random_user = x_train.getrow(random.randint(0, len(df.user_id.unique())))
prediction = model.predict(random_user)[0]
interacted_recipes = []
for recipe_id in random_user.indices:
    interacted_recipes.append(recipe_dict[recipe_id])
    prediction[recipe_id] = -100000


top_index = (-prediction).argsort()[:10]
recommended_recipes = []
for recipe_id in top_index:
    recommended_recipes.append(recipe_dict[recipe_id])

# get interacted recipes and recommended recipes
user_interactions = df_recipes[df_recipes['id'].isin(interacted_recipes)].drop('id', axis=1)
user_recommendations = df_recipes[df_recipes['id'].isin(recommended_recipes)].drop('id', axis=1)

display(user_interactions)
display(user_recommendations)
